# Units

yt has a robust symbolic unit system for use with computations. Units can be converted into others with equivalent dimensionality, and unit arithmetic is automatically carried through. 

As before, import yt. We'll also need to import NumPy.

In [ ]:
import yt
import numpy as np

Let's also load up a sample dataset for trying things out on:

In [ ]:
# Athena++ datasets don't have unit information built into them,
# so we have to tell them what units they are in 
units_override = {"length_unit": (1.0, "kpc"), "mass_unit": (1.0e14, "Msun"), "time_unit": (1.0, "Myr")}
ds = yt.load("AM06/AM06.out1.00500.athdf", units_override=units_override)

## Basic Information about Units and Unitful Arrays and Quantities

Unitful objects in yt are the `YTArray`, which is a NumPy array with unit information, and the `YTQuantity`, which is a scalar with units. We've seen `YTArray`s already, when we return fields from data objects:

In [ ]:
sp = ds.sphere("c", (200.0, "kpc"))
rho = sp['density'].copy() # Make a copy so we can manipulate it
print(rho)

`YTArrays` consist of a NumPy array and units:

In [ ]:
print(rho.value) # This returns a copy of the NumPy array
print(rho.view()) # This returns the actual NumPy array
print(rho.units) # This returns the unit object

Units also have a dimensions attribute:

In [ ]:
print(rho.units.dimensions)

`YTArrays` and `YTQuantities` can also be created from scratch using the `YTArray` and `YTQuantity` classes:

In [ ]:
from yt import YTArray, YTQuantity
a = YTArray(np.random.random(10), "kpc")
print(a)

In [ ]:
b = YTQuantity(1.0e-27, "erg/s/cm**2/steradian")
print(b)

### Unit Conversions

In [ ]:
print(rho.in_units('g/cm**3'))
print()
print(rho.in_units('Msun/kpc**3'))
print()
print(rho.in_units('lbm/ft**3')) # What's this? 

However, trying to convert a unitful object into units of different dimensions will have the (predictable) result:

In [ ]:
T = sp['temperature']
T.convert_to_units('kpc') # oops

### Arithmetic Operations with `YTArrays` and `YTQuantities`

Unitful objects with the same dimensions can be added and subtracted:

In [ ]:
rho + YTQuantity(200.0, "Msun/kpc**3")

In [ ]:
rho*sp["velocity_x"]

In [ ]:
np.sqrt(rho)

However, once again, don't do something silly!

In [ ]:
rho + sp["velocity_x"]

### Code Units

In [ ]:
for unit in ["length", "mass", "time", "velocity", "magnetic"]:
    print("code_%s =" % unit, getattr(ds, unit+"_unit"))

In [ ]:
v = ds.arr([1.0, 2.0, 3.0], "code_length/code_time")
print(v.in_units("km/s"))

In [ ]:
p = ds.quan(3.0e-3, "code_mass/code_time**2/code_length")
print(p.in_units("keV/cm**3"))

### Physical Constants

yt maintains a database of physical constants with units. They are `YTQuantities`! You can import them from `yt.units`, and use them in computations:

In [ ]:
from yt.units import G, kboltz, hbar
print(G)
print()
print(kboltz)
print()
print(hbar)

In [ ]:
print(kboltz*T)

## Unit Systems

The default set of units provided by yt are "Gaussian" cgs units, expressing all calculations in terms of centimeters, grams, seconds, Kelvin, radians, and derived units in terms of these. This means that most fields are returned in either these units, though some fields are returned in code units and some derived fields have special units.

As seen above, it is possible to convert these units into those from other systems, but yt also provides methods for converting arrays, quantities, and the fields for entire datasets into alternative unit systems.

First of all, simple methods are provided for conversion into MKSA and cgs units:

In [ ]:
# Methods for MKSA and cgs/Gaussian units
print(rho.in_mks())
print()
print(rho.in_cgs())

However, 

In [ ]:
yt.unit_system_registry['mks']

In [ ]:
yt.unit_system_registry['galactic']

In [ ]:
yt.unit_system_registry['imperial']

In [ ]:
yt.unit_system_registry['geometrized']

In [ ]:
print(rho.in_base('imperial'))
print()
print(rho.in_base('galactic'))
print()
print(rho.in_base('geometrized'))

## Equivalencies

Some quantities are not directly convertible to one another in yt (they don't have the same dimensions) but nevertheless are related via some physical relation or law. For this, yt provides a set of unit equivalencies.